In [1]:
import pandas as pd
import numpy as np
import requests
import time
import random
from tqdm import tqdm

In [2]:
cookies = r"_trackity=81022119-84ee-ed58-8c25-c4772e6b56f1; TOKENS={%22access_token%22:%22Jl5DrIsjp6kNc0RH1mMU4ZWhVSLbKfAy%22%2C%22expires_in%22:157680000%2C%22expires_at%22:1844943944403%2C%22guest_token%22:%22Jl5DrIsjp6kNc0RH1mMU4ZWhVSLbKfAy%22}; delivery_zone=Vk4wMzkwMDYwMDE=; _gid=GA1.2.634860621.1687263945; _gat=1; tiki_client_id=228222380.1687263945; _ga_GSD4ETCY1D=GS1.1.1687263948.1.0.1687263948.60.0.0; _ga=GA1.1.228222380.1687263945; _gcl_au=1.1.1639610846.1687263949; _fbp=fb.1.1687263948757.1893530334; _hjSessionUser_522327=eyJpZCI6IjcyYjczZjc2LWQ2N2YtNTM0NS05Yzg0LTJhMGI0MzM4OTU2NCIsImNyZWF0ZWQiOjE2ODcyNjM5NDg4MDEsImV4aXN0aW5nIjpmYWxzZX0=; _hjFirstSeen=1; _hjIncludedInSessionSample_522327=0; _hjSession_522327=eyJpZCI6IjFmZGJjYmU3LWYzNTctNDdlMi1hMmQxLWQ4MDA0MDU4ZjdkYSIsImNyZWF0ZWQiOjE2ODcyNjM5NDg4MDgsImluU2FtcGxlIjpmYWxzZX0=; _hjAbsoluteSessionInProgress=0; __uidac=05f3f93b0acfed74e61dab2be9997c21; amp_99d374=gX0IRGhoQ2geT8lKES4S-G...1h3cc97ta.1h3cc9cmt.2.5.7"

In [3]:
keys = []
values = []
for item in cookies.split('; '):
    item = item.split('=')
    keys.append(item[0])
    values.append(item[1])
cookies_dict = {key: value for key, value in zip(keys, values)}

headers_dict = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'en-US,en;q=0.9',
    'Referer': r'https://tiki.vn/ta-dan-huggies-skincare-super-jumbo-m76-6-p261996366.html?itm_campaign=CTP_YPD_TKA_PLA_UNK_ALL_UNK_UNK_UNK_UNK_X.252372_Y.1834692_Z.3771327_CN.LED_Category_Ta-Dan_Tram-Tra-%2F-LED_Category_Search-Ads&itm_medium=CPC&itm_source=tiki-ads&spid=261996367',
    'X-Guest-Token': 'Jl5DrIsjp6kNc0RH1mMU4ZWhVSLbKfAy',
    'Connection': 'keep-alive'
}

params_dict = {
    'platform': 'web',
    'spid': '261996367'
}

In [4]:
def parser(json):
    d = dict()
    try:
        d['id'] = json.get('id')
        d['quantity_sold'] = json.get('all_time_quantity_sold')
        d['has_freeship_plus_benefit'] = json.get('asa_cashback_widget').get('amplitude').get('has_freeship_plus_benefit')
        d['partner_rewards_amount'] = json.get('asa_cashback_widget').get('amplitude').get('partner_rewards_amount')
        d['tiki_rewards_amount'] = json.get('asa_cashback_widget').get('amplitude').get('tiki_rewards_amount')
        d['total_rewards_amount'] = json.get('asa_cashback_widget').get('amplitude').get('total_rewards_amount')
        d['store_id'] = json.get('current_seller').get('store_id')
        d['day_ago_created'] = json.get('day_ago_created')
        d['original_price'] = json.get('original_price')    
        d['price'] = json.get('price')
        d['discount'] = json.get('discount')
        d['discount_rate'] = json.get('discount_rate')
        d['gift_item_title'] = json.get('gift_item_title')
        d['has_buynow'] = json.get('has_buynow')
        d['fulfillment_type'] = json.get('inventory').get('fulfillment_type')
        d['inventory_status'] = json.get('inventory_status')    
        d['inventory_type'] = json.get('inventory_type')
        d['is_seller_in_chat_whitelist'] = json.get('is_seller_in_chat_whitelist')
        d['other_sellers'] = json.get('other_sellers')
        d['rating_average'] = json.get('rating_average')
        d['return_and_exchange_policy'] = json.get('return_and_exchange_policy')    
        d['review_count'] = json.get('review_count')
        d['type'] = json.get('type')
    except:
        d['id'] = np.nan
        d['quantity_sold'] = json.get('quantity_sold').get('value')
        d['has_freeship_plus_benefit'] = np.nan
        d['partner_rewards_amount'] = np.nan
        d['tiki_rewards_amount'] = np.nan
        d['total_rewards_amount'] = np.nan
        d['store_id'] = np.nan
        d['day_ago_created'] = np.nan
        d['original_price'] = np.nan  
        d['price'] = np.nan
        d['discount'] = np.nan
        d['discount_rate'] = np.nan
        d['gift_item_title'] = np.nan
        d['has_buynow'] = np.nan
        d['fulfillment_type'] = np.nan
        d['inventory_status'] = np.nan
        d['inventory_type'] = np.nan
        d['is_seller_in_chat_whitelist'] = np.nan
        d['other_sellers'] = np.nan
        d['rating_average'] = np.nan
        d['return_and_exchange_policy'] = np.nan
        d['review_count'] = np.nan
        d['type'] = np.nan
    return d

In [5]:
df_id = pd.read_csv('prod_id.csv').iloc[80000:100000]
pbar = tqdm(df_id.id.to_list(), total = len(df_id.id.to_list()))
result = []
for p_id in pbar:
    try:
        response = requests.get('https://tiki.vn/api/v2/products/{}'.format(p_id), 
                                headers = headers_dict, 
                                params = params_dict,
                                cookies = cookies_dict)
        if response.status_code == 200:
#            time.sleep(random.randrange(1, 3))
            pbar.set_description("Processing %s" % p_id)
            result.append(parser(response.json()))
    except:
        pass

Processing 94978634: 100%|██████████| 20000/20000 [1:55:49<00:00,  2.88it/s]    


In [6]:
result = [x for x in result if str(x) != 'nan']
df_prod = pd.DataFrame(result)
df_prod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5648 entries, 0 to 5647
Data columns (total 24 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           5648 non-null   int64  
 1   sku                          5648 non-null   object 
 2   quantity_sold                3747 non-null   float64
 3   has_freeship_plus_benefit    5648 non-null   bool   
 4   partner_rewards_amount       5648 non-null   int64  
 5   tiki_rewards_amount          5648 non-null   float64
 6   total_rewards_amount         5648 non-null   float64
 7   store_id                     5648 non-null   int64  
 8   day_ago_created              5648 non-null   int64  
 9   original_price               5648 non-null   int64  
 10  price                        5648 non-null   int64  
 11  discount                     5648 non-null   int64  
 12  discount_rate                5648 non-null   int64  
 13  gift_item_title   

In [7]:
df_prod.to_csv('df_prod1.csv', index = False)